In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import pprint
import json
import re

In [2]:
#user inputs airport codes
origination = input('Please enter the 3 letter airport code of the origination airport:').upper()
destination = input('Please enter the 3 letter airport code of the destination airport:').upper()

Please enter the 3 letter airport code of the origination airport: lax
Please enter the 3 letter airport code of the destination airport: sna


In [3]:
#read in airport codes lat/long file. Source: https://github.com/ip2location/ip2location-iata-icao/blob/master/iata-icao.csv
airport_codes = pd.read_csv('../data/airport_codes_.csv')
ac_df = pd.DataFrame(airport_codes)
ac_df.head()

,country_code,region_name,iata,icao,airport,latitude,longitude
0,AE,Abu Zaby,AAN,OMAL,Al Ain International Airport,24.2617,55.6092
1,AE,Abu Zaby,AUH,OMAA,Abu Dhabi International Airport,24.4330,54.6511
2,AE,Abu Zaby,AYM,NaN,Yas Island Seaplane Base,24.4670,54.6103
3,AE,Abu Zaby,AZI,OMAD,Al Bateen Executive Airport,24.4283,54.4581
4,AE,Abu Zaby,DHF,OMAM,Al Dhafra Air Base,24.2482,54.5477


In [4]:
#look up lat/long for airport codes
origin_lat = ac_df.loc[ac_df['iata'] == origination, ['latitude']].iloc[0,0]
origin_long = ac_df.loc[ac_df['iata'] == origination, ['longitude']].iloc[0,0]
dest_lat = ac_df.loc[ac_df['iata'] == destination, ['latitude']].iloc[0,0]
dest_long = ac_df.loc[ac_df['iata'] == destination, ['longitude']].iloc[0,0]


In [5]:
#get weather data. Source: https://weather-gov.github.io/api/general-faqs
origin_url = f'https://api.weather.gov/points/{origin_lat},{origin_long}'
dest_url = f'https://api.weather.gov/points/{dest_lat},{dest_long}'

# Origin 7 day forecast function

In [6]:
origination = input('Please enter the 3 letter airport code of the origination airport:').upper()

Please enter the 3 letter airport code of the origination airport: lax


In [7]:
date = str(input('Please enter date of flight in YYYY-MM-DD format:'))

Please enter date of flight in YYYY-MM-DD format: 2024-07-17


In [31]:
#define function for 7-day forecast for origin airport
def origin_fcstfn (date, origination):

    #read in airport codes lat/long file. Source: https://github.com/ip2location/ip2location-iata-icao/blob/master/iata-icao.csv
    airport_codes = pd.read_csv('data/airport_codes_.csv')
    ac_df = pd.DataFrame(airport_codes)
    
    #look up lat/long for airport codes
    origin_lat = ac_df.loc[ac_df['iata'] == origination, ['latitude']].iloc[0,0]
    origin_long = ac_df.loc[ac_df['iata'] == origination, ['longitude']].iloc[0,0]
    
    #get weather data. Source: https://weather-gov.github.io/api/general-faqs
    origin_url = f'https://api.weather.gov/points/{origin_lat},{origin_long}'

    #get 7-day forecast for origin
    origin_7dresponse = requests.get(origin_url).json()['properties']['forecast']
    origin_7dforecast = requests.get(origin_7dresponse).json()

    #find max temp for origin airport
    for i in range(len(origin_7dforecast['properties']['periods'])):
        start_time = origin_7dforecast['properties']['periods'][i]['startTime']
        match = re.search(f'^{date}', start_time)
        daytime = origin_7dforecast['properties']['periods'][i]['isDaytime']
        if match and daytime == True:
            origin_tmax = origin_7dforecast['properties']['periods'][i]['temperature']

    #find max wind speed for origin airport
    for i in range(len(origin_7dforecast['properties']['periods'])):
        start_time = origin_7dforecast['properties']['periods'][i]['startTime']
        match = re.search(f'^{date}', start_time)
        daytime = origin_7dforecast['properties']['periods'][i]['isDaytime']
        if match and daytime == True:
            try:
                day_awnd = int(origin_7dforecast['properties']['periods'][i]['windSpeed'].split(' ')[2])
            except:
                day_awnd = int(origin_7dforecast['properties']['periods'][i]['windSpeed'].split(' ')[0])
        else:
            try:
                night_awnd = int(origin_7dforecast['properties']['periods'][i]['windSpeed'].split(' ')[2])
            except:
                night_awnd = int(origin_7dforecast['properties']['periods'][i]['windSpeed'].split(' ')[0])
    origin_awnd = max(day_awnd, night_awnd)

    #find chance of precipitation at origination airport
    for i in range(len(origin_7dforecast['properties']['periods'])):
        start_time = origin_7dforecast['properties']['periods'][i]['startTime']
        match = re.search(f'^{date}', start_time)
        daytime = origin_7dforecast['properties']['periods'][i]['isDaytime']
        if match and daytime == True:
            day_precip = origin_7dforecast['properties']['periods'][i]['probabilityOfPrecipitation']['value']
            if day_precip == None:
                day_precip = 0
        else:
            night_precip = origin_7dforecast['properties']['periods'][i]['probabilityOfPrecipitation']['value']
            if night_precip == None:
                night_precip = 0
    origin_precip = max(day_precip, night_precip)


    return {
        'max_temp': origin_tmax,
        'max_wind_speed': origin_awnd,
        'chance_of_precipitation': f'{origin_precip}%'
    }

# Origin 7 day forecast

In [9]:
#get 7-day forecast for origin
origin_7dresponse = requests.get(origin_url).json()['properties']['forecast']
origin_7dforecast = requests.get(origin_7dresponse).json()
pprint.pp(origin_7dforecast)

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
              {'@version': '1.1',
               'wx': 'https://api.weather.gov/ontology#',
               'geo': 'http://www.opengis.net/ont/geosparql#',
               'unit': 'http://codes.wmo.int/common/unit/',
               '@vocab': 'https://api.weather.gov/ontology#'}],
 'type': 'Feature',
 'geometry': {'type': 'Polygon',
              'coordinates': [[[-118.4307614, 33.9467495],
                               [-118.42608589999999, 33.924526500000006],
                               [-118.39928269999999, 33.92840590000001],
                               [-118.40395299999999, 33.95062940000001],
                               [-118.4307614, 33.9467495]]]},
 'properties': {'units': 'us',
                'forecastGenerator': 'BaselineForecastGenerator',
                'generatedAt': '2024-07-17T05:00:10+00:00',
                'updateTime': '2024-07-17T04:49:38+00:00',
                'validTimes': '2024-07-1

In [10]:
#find max temp for origin airport
for i in range(len(origin_7dforecast['properties']['periods'])):
    start_time = origin_7dforecast['properties']['periods'][i]['startTime']
    match = re.search(f'^{date}', start_time)
    daytime = origin_7dforecast['properties']['periods'][i]['isDaytime']
    if match and daytime == True:
        origin_tmax = origin_7dforecast['properties']['periods'][i]['temperature']
origin_tmax

74

In [11]:
#find max wind speed for origin airport
for i in range(len(origin_7dforecast['properties']['periods'])):
    start_time = origin_7dforecast['properties']['periods'][i]['startTime']
    match = re.search(f'^{date}', start_time)
    daytime = origin_7dforecast['properties']['periods'][i]['isDaytime']
    if match and daytime == True:
        try:
            day_awnd = int(origin_7dforecast['properties']['periods'][i]['windSpeed'].split(' ')[2])
        except:
            day_awnd = int(origin_7dforecast['properties']['periods'][i]['windSpeed'].split(' ')[0])
    else:
        try:
            night_awnd = int(origin_7dforecast['properties']['periods'][i]['windSpeed'].split(' ')[2])
        except:
            night_awnd = int(origin_7dforecast['properties']['periods'][i]['windSpeed'].split(' ')[0])
origin_awnd = max(day_awnd, night_awnd)
origin_awnd

10

In [12]:
#find chance of precipitation at origination airport
for i in range(len(origin_7dforecast['properties']['periods'])):
    start_time = origin_7dforecast['properties']['periods'][i]['startTime']
    match = re.search(f'^{date}', start_time)
    daytime = origin_7dforecast['properties']['periods'][i]['isDaytime']
    if match and daytime == True:
        day_precip = origin_7dforecast['properties']['periods'][i]['probabilityOfPrecipitation']['value']
        if day_precip == None:
            day_precip = 0
    else:
        night_precip = origin_7dforecast['properties']['periods'][i]['probabilityOfPrecipitation']['value']
        if night_precip == None:
            night_precip = 0
origin_precip = max(day_precip, night_precip)
origin_precip

0

In [13]:
#forecast blurb at origin airport for origination airport
for i in range(len(origin_7dforecast['properties']['periods'])):
    start_time = origin_7dforecast['properties']['periods'][i]['startTime']
    match = re.search(f'^{date}', start_time)
    daytime = origin_7dforecast['properties']['periods'][i]['isDaytime']
    if match and daytime == True:
        origin_blurb = origin_7dforecast['properties']['periods'][i]['detailedForecast']
origin_blurb

'Patchy fog before 11am. Mostly sunny, with a high near 74. South southwest wind 5 to 10 mph.'

# Destination 7 day forecast

In [15]:
#get 7-day forecast for destination
dest_7dresponse = requests.get(dest_url).json()['properties']['forecast']
dest_7dforecast = requests.get(dest_7dresponse).json()
pprint.pp(dest_7dforecast)

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
              {'@version': '1.1',
               'wx': 'https://api.weather.gov/ontology#',
               'geo': 'http://www.opengis.net/ont/geosparql#',
               'unit': 'http://codes.wmo.int/common/unit/',
               '@vocab': 'https://api.weather.gov/ontology#'}],
 'type': 'Feature',
 'geometry': {'type': 'Polygon',
              'coordinates': [[[-117.8798383, 33.6822462],
                               [-117.8752822, 33.65999],
                               [-117.8485874, 33.6637727],
                               [-117.8531384, 33.6860294],
                               [-117.8798383, 33.6822462]]]},
 'properties': {'units': 'us',
                'forecastGenerator': 'BaselineForecastGenerator',
                'generatedAt': '2024-07-17T05:00:25+00:00',
                'updateTime': '2024-07-17T03:34:06+00:00',
                'validTimes': '2024-07-16T21:00:00+00:00/P7DT4H',
                'el

In [16]:
#find max temp for destination airport
for i in range(len(dest_7dforecast['properties']['periods'])):
    start_time = dest_7dforecast['properties']['periods'][i]['startTime']
    match = re.search(f'^{date}', start_time)
    daytime = dest_7dforecast['properties']['periods'][i]['isDaytime']
    if match and daytime == True:
        dest_tmax = dest_7dforecast['properties']['periods'][i]['temperature']
dest_tmax

80

In [17]:
#find max wind speed for destination airport
for i in range(len(dest_7dforecast['properties']['periods'])):
    start_time = dest_7dforecast['properties']['periods'][i]['startTime']
    match = re.search(f'^{date}', start_time)
    daytime = dest_7dforecast['properties']['periods'][i]['isDaytime']
    if match and daytime == True:
        try:
            day_awnd = int(dest_7dforecast['properties']['periods'][i]['windSpeed'].split(' ')[2])
        except:
            day_awnd = int(dest_7dforecast['properties']['periods'][i]['windSpeed'].split(' ')[0])
    else:
        try:
            night_awnd = int(dest_7dforecast['properties']['periods'][i]['windSpeed'].split(' ')[2])
        except:
            night_awnd = int(dest_7dforecast['properties']['periods'][i]['windSpeed'].split(' ')[0])
dest_awnd = max(day_awnd, night_awnd)
dest_awnd

10

In [18]:
#find chance of precipitation at destination airport
for i in range(len(dest_7dforecast['properties']['periods'])):
    start_time = dest_7dforecast['properties']['periods'][i]['startTime']
    match = re.search(f'^{date}', start_time)
    daytime = dest_7dforecast['properties']['periods'][i]['isDaytime']
    if match and daytime == True:
        day_precip = dest_7dforecast['properties']['periods'][i]['probabilityOfPrecipitation']['value']
        if day_precip == None:
            day_precip = 0
    else:
        night_precip = dest_7dforecast['properties']['periods'][i]['probabilityOfPrecipitation']['value']
        if night_precip == None:
            night_precip = 0
dest_precip = max(day_precip, night_precip)
dest_precip

0

In [19]:
#forecast blurb at origin airport for origination airport
for i in range(len(dest_7dforecast['properties']['periods'])):
    start_time = dest_7dforecast['properties']['periods'][i]['startTime']
    match = re.search(f'^{date}', start_time)
    daytime = dest_7dforecast['properties']['periods'][i]['isDaytime']
    if match and daytime == True:
        dest_blurb = dest_7dforecast['properties']['periods'][i]['detailedForecast']
dest_blurb

'Mostly sunny, with a high near 80. South wind 0 to 10 mph.'

# Origin hourly forecast - only day of

In [20]:
#get hourly forecast for origin
origin_response = requests.get(origin_url).json()['properties']['forecastHourly']
origin_forecast = requests.get(origin_response).json()
pprint.pp(origin_forecast)

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
              {'@version': '1.1',
               'wx': 'https://api.weather.gov/ontology#',
               'geo': 'http://www.opengis.net/ont/geosparql#',
               'unit': 'http://codes.wmo.int/common/unit/',
               '@vocab': 'https://api.weather.gov/ontology#'}],
 'type': 'Feature',
 'geometry': {'type': 'Polygon',
              'coordinates': [[[-118.4307614, 33.9467495],
                               [-118.42608589999999, 33.924526500000006],
                               [-118.39928269999999, 33.92840590000001],
                               [-118.40395299999999, 33.95062940000001],
                               [-118.4307614, 33.9467495]]]},
 'properties': {'units': 'us',
                'forecastGenerator': 'HourlyForecastGenerator',
                'generatedAt': '2024-07-17T05:00:36+00:00',
                'updateTime': '2024-07-17T04:49:38+00:00',
                'validTimes': '2024-07-16T

In [21]:
# find temp max for the day at origin airport
origin_tmax = 0
for i in range(len(origin_forecast['properties']['periods'])):
    if origin_forecast['properties']['periods'][i]['temperature'] > origin_tmax: \
        origin_tmax = origin_forecast['properties']['periods'][i]['temperature'] 
    else: 
        origin_tmax = origin_tmax
origin_tmax

76

In [22]:
#find max % chance of precipitation for day at origin airport
origin_precip = 0
for i in range(len(origin_forecast['properties']['periods'])):
    precip = origin_forecast['properties']['periods'][i]['probabilityOfPrecipitation']['value']
    if precip == None:
        precip = 0
    else: 
        precip = precip
    if precip > origin_precip:
        origin_precip = precip
    else: 
        origin_precip = origin_precip
origin_precip

3

# Destination hourly forecast - only day of

In [23]:
#get hourly forecast for destination
dest_response = requests.get(dest_url).json()['properties']['forecastHourly']
dest_forecast = requests.get(dest_response).json()
pprint.pp(dest_forecast)

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
              {'@version': '1.1',
               'wx': 'https://api.weather.gov/ontology#',
               'geo': 'http://www.opengis.net/ont/geosparql#',
               'unit': 'http://codes.wmo.int/common/unit/',
               '@vocab': 'https://api.weather.gov/ontology#'}],
 'type': 'Feature',
 'geometry': {'type': 'Polygon',
              'coordinates': [[[-117.8798383, 33.6822462],
                               [-117.8752822, 33.65999],
                               [-117.8485874, 33.6637727],
                               [-117.8531384, 33.6860294],
                               [-117.8798383, 33.6822462]]]},
 'properties': {'units': 'us',
                'forecastGenerator': 'HourlyForecastGenerator',
                'generatedAt': '2024-07-17T05:00:36+00:00',
                'updateTime': '2024-07-17T03:34:06+00:00',
                'validTimes': '2024-07-16T21:00:00+00:00/P7DT4H',
                'elev

In [24]:
# find temp max for the day at destination airport
dest_tmax = 0
for i in range(len(dest_forecast['properties']['periods'])):
    if dest_forecast['properties']['periods'][i]['temperature'] > dest_tmax: \
        dest_tmax = origin_forecast['properties']['periods'][i]['temperature'] 
    else: 
        dest_tmax = dest_tmax
dest_tmax

71

In [25]:
# find max wind speed for the day at origin airport
dest_awnd = 0
for i in range(len(dest_forecast['properties']['periods'])):
    awnd = int(dest_forecast['properties']['periods'][i]['windSpeed'].split(' ')[0])
    if awnd > dest_awnd:
        dest_awnd = awnd
    else:
        dest_awnd = awnd
dest_awnd

0

In [26]:
#find max % chance of precipitation for day at dest airport
dest_precip = 0
for i in range(len(origin_forecast['properties']['periods'])):
    precip = origin_forecast['properties']['periods'][i]['probabilityOfPrecipitation']['value']
    if precip == None:
        precip = 0
    else: 
        precip = precip
    if precip > dest_precip:
        dest_precip = precip
    else: 
        dest_precip = dest_precip
dest_precip

3